<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Data_Augmentation_and_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset.7z
!7z x /content/Dataset.7z

--2021-03-17 04:46:35--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22159364 (21M) [application/octet-stream]
Saving to: ‘Dataset.7z’

Dataset.7z          100%[===================>]  21.13M   107MB/s    in 0.2s    

2021-03-17 04:46:36 (107 MB/s) - ‘Dataset.7z’ saved [22159364/22159364]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 22159364 bytes (22 MiB)

Extracting archive: /content/Dataset.7z
--
Path = /content/Data

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import torchvision
from PIL import Image

In [78]:
data = pd.read_excel("/content/Dataset/all_meta_data.xlsx")

In [79]:
data['phylum'] = data['phylum'].replace({'Roundworms':'Nematoda','Flatworm':'Platyhelminthes'})
data['class'] = data['class'].replace({'\tConoidasida':'Conoidasida','Tapeworms':'Cestoda'})
data['species'] = data['species'].replace({'Ancyclostoma sp.':'Ancyclostoma sp.','Entamoeba histolytica]':'Entamoeba histolytica'})
data['form'] = data['form'].replace({'rings':'trophozoites','decorticated egg':'egg'})
data['sample'] = data['sample'].replace({'intestinal tissue':'intestine', 'intestine':'intestine'})

In [80]:
data = [x for _,x in data.groupby('species') if len(x)>12]

data = pd.concat(data,ignore_index=True)
data['species'] = data['species'].str.replace('.','')


In [77]:
!rm -r train val

In [65]:
data['species'].value_counts()

Plasmodium malariae          37
Cyclospora cayetanensis      37
Entamoeba histolytica        36
Ascaris lumbricoides         36
Plasmodium vivax             36
Giardia duodenalis           36
Plasmodium falciparum        36
Plasmodium ovale             34
Strongyloides stercoralis    22
Dibothriocephalus latus      21
Trichuris Trichuria          19
Enterobius vermicularis      18
Toxocara canis               17
Ancyclostoma sp              17
Sarcocystis sp               16
Cryptosporidium sp           16
Plasmodium knowlesi          15
Hymenolepis nana             14
Toxoplasma gondii            13
Trichinella sp               13
Name: species, dtype: int64

In [81]:
train, val = train_test_split(
    data, test_size=0.25, random_state=2021,stratify = data['species'])

In [82]:

for path in data['species'].unique():
  os.makedirs(f"/content/train/{path}")


for path in data['species'].unique():
  os.makedirs(f"/content/val/{path}")

In [69]:
t = []
t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomPerspective(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomVerticalFlip(p=1)]))


t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomVerticalFlip(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomPerspective(p=1)]))

In [88]:
def transform_save(data,index,folder):
  img = Image.open(data[8])
  images = [img]
  images.extend((T(img) for T in t))
  for i, image in enumerate(images):
    image.convert('RGB').save(f"/content/{folder}/{data[3]}/T_{i}_{index}.jpg")
  values = [[data[0],data[1],data[2],data[3],data[4],data[5],f"/content/{data[3]}/T_{i}_{index}.jpg"] for i in range(len(images))]
  return values

def get_dataframe(dataframe,folder = 'train'):
  new = []
  for i in dataframe.index:
    new.extend(transform_save(dataframe.loc[i,:].values,index =i,folder=folder))
  return pd.DataFrame(new,columns=['phylum','class','genus','species','form','sample','img_path'])

In [89]:
train = get_dataframe(train)

In [90]:
val = get_dataframe(val,folder = 'val')

In [91]:
train.to_csv('/content/train/train.csv',index = False)
val.to_csv('/content/val/val.csv',index = False)

In [92]:
!7z a Train.7z train
!7z a Val.7z val


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           21 folders, 1831 files, 23281291 bytes (23 MiB)

Creating archive: Train.7z

Items to compress: 1852

  0%      5% 171 + train/Ascaris lumbricoides/T_3_51.jpg                                                 11% 301 + train/Cyclospora cayetanensis/T_1_82.jpg                                                    18% 433 + train/Dibothriocephalus latus/T_2_108.jpg                                                     24% 508 + train/Ent